Canevas de code python pour le DM décarbonation.
Ce document vous fournit :
    une liste des modules pythons à importer
    un modèle pour l’import de table de données formatées en .csv selon les habitudes françaises
    les paramètres permettant d’écrire les étiquettes de temps sur l’axe d’une figure avec un pas de temps mensuel sous la forme «Mois»
    un canevas de tracé de  courbes de données indexées sur le temps, avec titres et étiquettes, ainsi qu’une fonction d’export.

@author: Martin Hennebel

In [3]:
# modules utiles à importer
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates
import locale #pour paramètre locaux de date

!sudo apt-get update -y
!sudo apt-get install -y locales
!sudo locale-gen fr_FR.UTF-8
!sudo update-locale LANG=fr_FR.UTF-8

!locale -a

locale.setlocale(locale.LC_ALL, 'fr_FR.utf8')


Get:1 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Hit:2 http://archive.ubuntu.com/ubuntu noble InRelease                         
Hit:3 https://apt.postgresql.org/pub/repos/apt noble-pgdg InRelease            
Get:4 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]        
Get:5 http://archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]      
Hit:6 https://ppa.launchpadcontent.net/git-core/ppa/ubuntu noble InRelease
Ign:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu noble InRelease
Err:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu noble Release
  404  Not Found [IP: 185.125.190.80 443]
Reading package lists... Done
E: The repository 'https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu noble Release' does not have a Release file.
N: Updating from such a repository can't be done securely, and is therefore disabled by default.
N: See apt-secure(8) manpage for repository creation and user configuration details.
W: https:

'fr_FR.utf8'

In [4]:
from datetime import datetime
print(datetime.now().strftime("%A %d %B %Y"))
print(locale.format_string("%.2f", 12345.6789, grouping=True))

vendredi 07 novembre 2025
12 345,68


In [5]:
#%% liens vers les docs de fonctions pandas utiles 
"""
10 minutes d’introduction à pandas
https://pandas.pydata.org/docs/user_guide/10min.html
Un tutoriel pour débuter avec les séries temporelles dans pandas :
https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html    
Notice pour traiter les séries temporelles avec pandas
https://pandas.pydata.org/docs/user_guide/timeseries.html

Notices des fonctions pandas permettant de traiter des tables de données «Dataframe»
https://pandas.pydata.org/docs/reference/frame.html
    
parmi ces fonctions certaines à regarder :
pandas.DataFrame.index      pour récupérer l’index des lignes
pandas.DataFrame.columns    pour récupérer les titres des colonnes    
pandas.DataFrame.dtypes     pour récupérer les types de données dans la table
pandas.DataFrame.loc        pour extraire des données localisées par leurs étiquettes
pandas.DataFrame.iloc       pour extraire des données localisées par leurs positions (nombres entiers)
pandas.DataFrame.max        pour obtenir le maximum (min pour le minimum)
pandas.DataFrame.idxmax     pour obtenir l’indice du maximum (min pour le minimum)
pandas.DataFrame.nlargest   pour obtenir les n plus grandes valeurs (nsmallest pour les plus petites)
pandas.DataFrame.to_timestamp()   pour transformer des index en format temporel
pandas.DataFrame.tz_convert()    pour convertir un index temporel d’un fuseau horaire à un autre (Continental European Time pour Paris)
pandas.DataFrame.tz_localize()    pour affecter un fuseau horaire à un index temporel naïf
pandas.to_datetime              pour convertir des arguments en date + heure (datetime)
pandas.DataFrame.sort_values    pour trier les valeurs par ordre

…
"""

'\n10 minutes d’introduction à pandas\nhttps://pandas.pydata.org/docs/user_guide/10min.html\nUn tutoriel pour débuter avec les séries temporelles dans pandas :\nhttps://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html    \nNotice pour traiter les séries temporelles avec pandas\nhttps://pandas.pydata.org/docs/user_guide/timeseries.html\n\nNotices des fonctions pandas permettant de traiter des tables de données «Dataframe»\nhttps://pandas.pydata.org/docs/reference/frame.html\n\nparmi ces fonctions certaines à regarder :\npandas.DataFrame.index      pour récupérer l’index des lignes\npandas.DataFrame.columns    pour récupérer les titres des colonnes    \npandas.DataFrame.dtypes     pour récupérer les types de données dans la table\npandas.DataFrame.loc        pour extraire des données localisées par leurs étiquettes\npandas.DataFrame.iloc       pour extraire des données localisées par leurs positions (nombres entiers)\npandas.DataFrame.max        pour obtenir le m

In [ ]:
#%% Modèle d’import de données en csv formaté à la française (; en séparateur, virgule comme séparteur décimal, formatage des dates)

#donnees_importees_Pays_annee=pd.read_csv('Table_Donnees.csv',sep=';',decimal=',',index_col=[0],parse_dates=[0],date_format="ISO8601")
#%% paramètres des étiquettes temporelles pour l’axe du temps dans les figures. 
  # intervalle mensuel pour les étiquettes

#Mois=mdates.MonthLocator(bymonthday=1)# chaque mois, le premier jour du mois.

  #format affichage Mois : 
#MoisFmt = mdates.DateFormatter('%b')# nom du mois abrégé sur 4 lettres.
#Hours = mdates.HourLocator()   #

#%%Template de Tracé d’une figure
  #On crée le repère        
#fig, ax = plt.subplots()
  #tracé de la courbe conso vs temps, avec des valeurs de divisées par 1000, pour une table de données indexée sur le temps.
#ax.plot(Table_Donnees.index,(Table_Donnes['Nom_Colonne_Donnees'].values/1000), linewidth=.5)

   #affichage du titre
#plt.title('Titre de la figure')
#Limites min et max de l’axe x
#ax.set_xlim(Table_Donnees.index[0],Table_Donnees.index[-1])

#positionnement des étiquettes de temps à chaque mois
#ax.xaxis.set_major_locator(Mois)
#format des étiquettes de temps
#ax.xaxis.set_major_formatter(MoisFmt)
#légende de l'axe des abscisses
#plt.xlabel('Mois')

#Limites min max de l’axe y
#ax.set_ylim(0,100)
#légende de l'axe des ordonnées
#plt.ylabel('Valeurs (Unité)')


# affichage de la grille pour faciliter la lecture des valeurs
#plt.grid(True)
# affichage de la figure
#plt.show()
#sauvegarde de la figure en .svg :
#plt.savefig(r'Adresse\du\Repertoire\' + 'Nom_Fichier.svg')
#les enregistrement de figure doivent être fait en format vectoriel : .svg, .emf, .eps
# Éviter les enregistrements en format matriciels, lourds et flou en agrandissement : .bmp, .jpg …

## Exercice 1 Analyse de la consommation
Importer la série temporelle des consommations au pas horaire de l’année 2024 (fichier Don-
nees_France_2024.csv)

In [10]:

df = pd.read_csv("Donnees_France_2024_fr_cet.csv", sep=';') # on ajoute le séparateur car on voit en printant head que c'est ce qui est utilisé
print(df.head())

                       Index   Load  Biomass  Fossil Gas  Fossil Hard coal  \
0  2024-01-01 00:00:00+01:00  52847      343        1810              15.0   
1  2024-01-01 01:00:00+01:00  51600      343        1840              16.0   
2  2024-01-01 02:00:00+01:00  50992      343        1787              16.0   
3  2024-01-01 03:00:00+01:00  48187      343        1778              15.0   
4  2024-01-01 04:00:00+01:00  46479      343        1765              15.0   

   Fossil Oil  Hydro Pumped Storage Generation  \
0         162                              NaN   
1         191                              NaN   
2         190                              NaN   
3         190                              NaN   
4         191                              NaN   

   Hydro Pumped Storage Consumption  Hydro Run-of-river and poundage  \
0                            2033.0                             4822   
1                            2067.0                             4767   
2             

In [14]:
# question 1 : On cherche le nombre de valeurs, on va regarder le nombre de colonnes et le nombre de ligne
print(df.shape)
print(24*365)

(8784, 15)
8760


Il y a 8784 lignes pour 15 colonnes. 
Il y a donc 8784 valeurs pour chaque catégorie. 
On note qu'il y a des valeurs manquantes Nan. On peut les compter dans un second temps.

Il y a 24x365h dans une année soit : 8760. On remarque donc que c'est une année +1 jour.
Ce qui est cohérent car l'année 2024 est une année bissextile !
Il y a donc 366 jours soit 8784 heures. On retrouve le nombre de valeurs.